# Step 11 가변 길이 인수(순전파 편)
## 11.1 Function 클래스 추정

In [1]:
import numpy as np

class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None
  
  def set_creator(self, func):
    self.creator = func

  def backward(self):
    funcs = [self.creator]     
    while funcs :              
      f = funcs.pop()          
      x, y = f.input, f.output 
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        funcs.append(x.creator) 

In [8]:
def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

class Function:
  '''
    __call__ 에서 인수로 이제 list나 tuple 등의 가변 길이 인수를 받는다.
    받은 입력에 대해서 각각의 인자에 forward를 수행하고, set_creator를 한다.
  '''
  def __call__(self, inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(xs)
    outputs = [Variable(as_array(y)) for y in ys]
    
    for output in outputs:
      output.set_creator(self) 
    
    self.input = input
    self.output= output 
    return outputs

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

In [10]:
class Add(Function):
  def forward(self, xs):
    x0, x1 = xs
    y = x0 + x1
    return (y,) # tuple 형태로 반환

In [11]:
xs = [Variable(np.array(2)), Variable(np.array(3))] # 리스트 형태로 인수를 받아봄.
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5
